In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Load 'train.csv'**
train.csv 的資料為 12 個月中，每個月取 20 天，每天 24 小時的資料(每小時資料有 18 個 features)。

In [3]:
# data = pd.read_csv('gdrive/My Drive/hw1-regression/train.csv', header = None, encoding = 'big5')
# data = pd.read_csv('/kaggle/input/ml2020spring-hw1/train.csv', encoding = 'big5')
data_path = 'data/train.csv'
data = pd.read_csv(data_path)

In [4]:
# data.iloc[9]
# data.iloc[:17]

# **Preprocessing** 
取需要的數值部分，將 'RAINFALL' 欄位全部補 0。Rainfall属性的无降雨值为NR，降雨值为1. 将NR替换为数值0
另外，如果要在 colab 重覆這段程式碼的執行，請從頭開始執行(把上面的都重新跑一次)，以避免跑出不是自己要的結果（若自己寫程式不會遇到，但 colab 重複跑這段會一直往下取資料。意即第一次取原本資料的第三欄之後的資料，第二次取第一次取的資料掉三欄之後的資料，...）。

In [5]:
# 去除前三列(日期/测站/测项)
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
#pandas to_numpy（）产生一个列表数组

In [6]:
# print(raw_data)

# **Extract Features (1)**
属性组成18维向量
將原始 4320 * 18 的資料依照每個月分重組成 12 個月( 每个月20day*24h=480h横向小时 ), 18 (features) * 480 (hours) 的資料。
这样,每个月的资料在横向上是连续的时间

In [7]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
        #取raw_data的行,对sample的列进行拼接
    month_data[month] = sample

In [8]:
# print(month_data[0][0])

# **Extract Features (2)**
每个array是一个月, print(month_data[0][0])可查看第一个月第一天
每個月會有 480hrs，每 9 小時形成一個 data，每個月會有 471 個 data，故總資料數為 471 * 12 筆，而每筆 data 有 9 * 18 的 features (一小時 18 個 features * 9 小時)。

對應的 target 則有 471 * 12 個(第 10 個小時的 PM2.5)

前9h数据作为train_x, 第10h的pm2.5作为train_y, 制作xy样本

In [9]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:#最后一天的最后10h是最后一组样本了,再往后无法作样本了
                continue
            #-1在这里应该可以理解为一个正整数通配符，它代替任何整数. reshape(1, -1)化为一行
            # print(month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1))
            #x分片时是所有行(属性), 如果不进行reshape, 则每个属性的9小时单独成1行
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value第10个属性是PM2.5
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


In [10]:
print(f"x_shape = {x.shape}, y_shape = {y.shape}")

x_shape = (5652, 162), y_shape = (5652, 1)


# **Normalize (1)**
预处理数据有一个很关键的步骤就是数据的标准化
许多学习算法中目标函数的基础都是假设所有的特征都是零均值并且具有同一阶数上的方差。如果某个特征的方差比其他特征大几个数量级，那么它就会在学习算法中占据主导位置，导致学习器并不能像我们说期望的那样，从其他特征中学习。
## 1 StandardScaler
标准化数据通过减去均值然后除以方差（或标准差），这种数据标准化方法经过处理后数据符合标准正态分布，即均值为0，标准差为1
## 2 MinMaxScaler
这种方法是对原始数据的线性变换，将数据归一到[0,1]中间
## 3 RobustScaler
如果你的数据包含许多异常值，使用均值和方差缩放可能并不是一个很好的选择。
This Scaler removes the median（中位数） and scales the data according to the quantile range(四分位距离，也就是说排除了outliers)

In [11]:
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 标准差
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]

In [12]:
x

array([[-1.35825331, -1.35883937, -1.359222  , ...,  0.26650729,
         0.2656797 , -1.14082131],
       [-1.35825331, -1.35883937, -1.51819928, ...,  0.26650729,
        -1.13963133, -1.32832904],
       [-1.35825331, -1.51789368, -1.67717656, ..., -1.13923451,
        -1.32700613, -0.85955971],
       ...,
       [-0.88092053, -0.72262212, -0.56433559, ..., -0.57693779,
        -0.29644471, -0.39079039],
       [-0.7218096 , -0.56356781, -0.72331287, ..., -0.29578943,
        -0.39013211, -0.1095288 ],
       [-0.56269867, -0.72262212, -0.88229015, ..., -0.38950555,
        -0.10906991,  0.07797893]])

In [13]:
x.shape

(5652, 162)

# **Split Training Data Into "train_set" and "validation_set"**
這部分是針對作業中第二題、第三題，生成比較中用來訓練的 train_set 和不會被放入訓練、只是用來驗證的 validation_set。

In [10]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]#取80%的样本, floor向下取整. 0~0.8
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]#0.8~1
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

In [11]:
# # 题目4-数据划分-只取后5h-cell
# #只能运行一次
# x_5h = x[:, (4*18):]
# #test测试集数据预处理
# testdata = pd.read_csv('/kaggle/input/ml2020spring-hw1/test.csv', header = None, encoding = 'big5')
# test_data = testdata.iloc[:, 2:]#只能运行一次
# test_data[test_data == 'NR'] = 0
# test_data = test_data.to_numpy()

In [12]:
# # 题目4-测试数据形状改变-只取后5h-cell
# test_x = np.empty([240, 18*9], dtype = float)
# # test_x_5h = np.empty([240, 18*5], dtype = float)
# # 10 小時為一筆，前九小時作x的feature. 一共取出 240 筆不重複的 test data，
# for i in range(240):#每一笔分别存, 一笔中有18个feature
#     test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)#原数据的小时在列中体现, 把所有前9小时都存入了
# #     test_x_5h[i, :] = test_data[18 * i: 18* (i + 1), 4:].reshape(1, -1)#只取后5h
# test_x_5h = test_x[:, (4*18):]

In [13]:
# # 题目4-模型改进-只取后5h-cell
# dim_5h = 18 * 5 + 1
# w_5h = np.zeros([dim_5h, 1])
# n_train_set = math.floor(12 * 471)
# #train
# learning_rate = 2
# iter_time = 1000
# eps = 0.0000000001
# #np.ones全部赋为1, concatenate数组拼接
# x_5h = np.concatenate((np.ones([n_train_set, 1]), x_5h), axis = 1).astype(float)
# #train
# adagrad_5h = np.zeros([dim_5h, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss = np.sqrt(np.sum(np.power(np.dot(x_5h, w_5h) - y, 2))/n_train_set)#rmse标准差
#     #每进行100次迭代, 输出loss值
#     if(t%100==0):
#         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient_5h = 2 * np.dot(x_5h.transpose(), np.dot(x_5h, w_5h) - y) #dim*1
#     adagrad_5h += gradient_5h ** 2
#     w_5h = w_5h - learning_rate * gradient_5h / np.sqrt(adagrad_5h + eps)
# np.save('weight_5h.npy', w_5h)
# w_5h

In [14]:
# # 题目4-测试集预测-只取后5h-cell
# #测试集输入数据标准化
# mean_x_5h = np.mean(test_x_5h, axis = 0)
# std_x_5h = np.std(test_x_5h, axis = 0) #标准差
# for i in range(len(test_x_5h)):
#     for j in range(len(test_x_5h[0])):
#         if std_x_5h[j] != 0:
#             test_x_5h[i][j] = (test_x_5h[i][j] - mean_x_5h[j]) / std_x_5h[j]
# test_x_5h = np.concatenate((np.ones([240, 1]), test_x_5h), axis = 1).astype(float)

In [15]:
# # 题目4-测试集预测-只取后5h-cell
# #预测
# w_5h = np.load('weight_5h.npy')
# ans_y_5h = np.dot(test_x_5h, w_5h)
# # ans_y_5h
# #保存预测数据
# import csv
# with open('submit.csv', mode='w', newline='') as submit_file:
#     csv_writer = csv.writer(submit_file)
#     header = ['id', 'value']
#     print(header)
#     csv_writer.writerow(header)
#     for i in range(240):
#         row = ['id_' + str(i), ans_y_5h[i][0]]
#         csv_writer.writerow(row)
#         print(row)

In [16]:
# # 题目3-PM2.5输入特征划分-cell
# n_train_set = math.floor(12 * 471 * 0.8)
# n_validation_set = math.ceil(12 * 471 * 0.2)

# # x = np.empty([12 * 471, 18 * 9], dtype = float)
# x_train_set_PM = np.empty([n_train_set, 1 * 9], dtype = float)
# x_validation_PM = np.empty([n_validation_set, 1 * 9], dtype = float)
# for hour in range(9):#每笔输入数据中的小时9, 和提取PM2.5所在的行数9, 在理解的时候不要搞混
#     x_train_set_PM[:,hour] = x_train_set[:, 18*hour + 9]
#     x_validation_PM[:,hour] = x_validation[:, 18*hour + 9]
# x_train_set_PM, x_validation_PM

In [17]:
# # 题目3-全属性对比只输入PM2.5数据-cell
# # 训练
# dim = 18 * 9 + 1
# w = np.zeros([dim, 1])
# x_train_set = np.concatenate((np.ones([n_train_set, 1]), x_train_set), axis = 1).astype(float)
# # x_train_set
# #train
# learning_rate = 100
# iter_time = 1000
# adagrad = np.zeros([dim, 1])
# eps = 0.0000000001
# for t in range(iter_time):
#     loss = np.sqrt(np.sum(np.power(np.dot(x_train_set, w) - y_train_set, 2))/n_train_set)#rmse标准差也要根据划分后的数据笔数修改所除的n
#     #每进行100次迭代, 输出loss值
# #     if(t%100==0):
# #         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x_train_set.transpose(), np.dot(x_train_set, w) - y_train_set) #dim*1
#     adagrad += gradient ** 2
#     w = w - learning_rate * gradient / np.sqrt(adagrad + eps)

In [18]:
# # 题目3-cell
# # PM2.5训练
# dim_PM = 1 * 9 + 1
# w_PM = np.zeros([dim_PM, 1])
# #np.ones全部赋为1, concatenate数组拼接
# x_train_set_PM = np.concatenate((np.ones([n_train_set, 1]), x_train_set_PM), axis = 1).astype(float)
# #train
# adagrad_PM = np.zeros([dim_PM, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss = np.sqrt(np.sum(np.power(np.dot(x_train_set_PM, w_PM) - y_train_set, 2))/n_train_set)#rmse标准差
#     #每进行100次迭代, 输出loss值
# #     if(t%100==0):
# #         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient_PM = 2 * np.dot(x_train_set_PM.transpose(), np.dot(x_train_set_PM, w_PM) - y_train_set) #dim*1
#     adagrad_PM += gradient_PM ** 2
#     w_PM = w_PM - learning_rate * gradient_PM / np.sqrt(adagrad_PM + eps)
    
    
# # 测试
# x_validation = np.concatenate((np.ones([n_validation_set, 1]), x_validation), axis = 1).astype(float)
# ans_y = np.dot(x_validation, w)
# # PM2.5
# x_validation_PM = np.concatenate((np.ones([n_validation_set, 1]), x_validation_PM), axis = 1).astype(float)
# ans_y_PM = np.dot(x_validation_PM, w_PM)


# # 分别计算平均误差
# n_vali=len(ans_y)
# average_error=0
# for i in range(0, n_vali):  
#     average_error+=abs(y_validation[i][0]-ans_y[i][0])
# average_error/=n_vali
# # PM2.5
# average_error_PM=0
# for i in range(0, n_vali):  
#     average_error_PM+=abs(y_validation[i][0]-ans_y_PM[i][0])
# average_error_PM/=n_vali

# print(average_error)
# print(average_error_PM)

In [19]:
# # 题目2-5小时输入划分-cell
# x_train_set_5h = x_train_set[:, (4*18):]
# x_validation_5h = x_validation[:, (4*18):]

In [20]:
# #题目2-5h,9h对比数据训练-cell
# dim = 18 * 9 + 1
# n_train_set = math.floor(12 * 471 * 0.8)
# w = np.zeros([dim, 1])
# x_train_set = np.concatenate((np.ones([n_train_set, 1]), x_train_set), axis = 1).astype(float)
# # x_train_set
# #train
# learning_rate = 100
# iter_time = 1000
# adagrad = np.zeros([dim, 1])
# eps = 0.0000000001
# for t in range(iter_time):
#     loss = np.sqrt(np.sum(np.power(np.dot(x_train_set, w) - y_train_set, 2))/n_train_set)#rmse标准差也要根据划分后的数据笔数修改所除的n
#     #每进行100次迭代, 输出loss值
#     if(t%100==0):
#         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x_train_set.transpose(), np.dot(x_train_set, w) - y_train_set) #dim*1
#     adagrad += gradient ** 2
#     w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
# # 5h
# dim_5h = 18 * 5 + 1
# w_5h = np.zeros([dim_5h, 1])
# #np.ones全部赋为1, concatenate数组拼接
# x_train_set_5h = np.concatenate((np.ones([n_train_set, 1]), x_train_set_5h), axis = 1).astype(float)
# #train
# adagrad_5h = np.zeros([dim_5h, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss = np.sqrt(np.sum(np.power(np.dot(x_train_set_5h, w_5h) - y_train_set, 2))/n_train_set)#rmse标准差
#     #每进行100次迭代, 输出loss值
# #     if(t%100==0):
# #         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient_5h = 2 * np.dot(x_train_set_5h.transpose(), np.dot(x_train_set_5h, w_5h) - y_train_set) #dim*1
#     adagrad_5h += gradient_5h ** 2
#     w_5h = w_5h - learning_rate * gradient_5h / np.sqrt(adagrad_5h + eps)

In [21]:
# #题目2-5h,9h对比数据测试-cell
# #test
# x_validation = np.concatenate((np.ones([math.ceil(12 * 471 * 0.2), 1]), x_validation), axis = 1).astype(float)
# # test_x
# # w = np.load('weight-by-train-set.npy')
# ans_y = np.dot(x_validation, w)
# # ans_y
# # 5h
# x_validation_5h = np.concatenate((np.ones([math.ceil(12 * 471 * 0.2), 1]), x_validation_5h), axis = 1).astype(float)
# ans_y_5h = np.dot(x_validation_5h, w_5h)
# ans_y_5h

In [22]:
# #题目2-分别计算平均误差-cell
# n_vali=len(ans_y)
# average_error=0
# for i in range(0, n_vali):  
#     average_error+=abs(y_validation[i][0]-ans_y[i][0])
# average_error/=n_vali
# # 5h
# average_error_5h=0
# for i in range(0, n_vali):  
#     average_error_5h+=abs(y_validation[i][0]-ans_y_5h[i][0])
# average_error_5h/=n_vali

# print(average_error)
# print(average_error_5h)

# **Training**

下面的 code 採用 Root Mean Square Error

因為常數項的存在，所以 dimension (dim) 需要多加一欄；eps 項是避免 adagrad 的分母為 0 而加的極小數值。

每一個 dimension (dim) 會對應到各自的 gradient, weight (w)，透過一次次的 iteration (iter_time) 學習。

In [23]:
# baseline-cell
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
#np.ones全部赋为1, concatenate数组拼接
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
x

array([[ 1.        , -1.35825331, -1.35883937, ...,  0.26650729,
         0.2656797 , -1.14082131],
       [ 1.        , -1.35825331, -1.35883937, ...,  0.26650729,
        -1.13963133, -1.32832904],
       [ 1.        , -1.35825331, -1.51789368, ..., -1.13923451,
        -1.32700613, -0.85955971],
       ...,
       [ 1.        , -0.88092053, -0.72262212, ..., -0.57693779,
        -0.29644471, -0.39079039],
       [ 1.        , -0.7218096 , -0.56356781, ..., -0.29578943,
        -0.39013211, -0.1095288 ],
       [ 1.        , -0.56269867, -0.72262212, ..., -0.38950555,
        -0.10906991,  0.07797893]])

In [24]:
# #题目1-作图cell
# learning_rate = [0.5, 2, 50, 100]
# iter_time = 1000
# adagrad = np.zeros([dim, 1])
# eps = 0.0000000001
# model_data = {
#     't': [ ],
#     'loss1': [ ],
#     'loss2': [ ],
#     'loss3': [ ],
#     'loss4': [ ]
# }
# w1=w#因为作图时不同学习率都要用到w,adagrad,需要留底或重置
# w2=w
# w3=w
# w4=w
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss1 = np.sqrt(np.sum(np.power(np.dot(x, w1) - y, 2))/471/12)#rmse标准差
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x.transpose(), np.dot(x, w1) - y) #dim*1
#     adagrad += gradient ** 2
#     w1 = w1 - learning_rate[0] * gradient / np.sqrt(adagrad + eps)
#     model_data['t'].append(t)
#     model_data['loss1'].append(loss1)
# adagrad = np.zeros([dim, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss2 = np.sqrt(np.sum(np.power(np.dot(x, w2) - y, 2))/471/12)#rmse标准差
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x.transpose(), np.dot(x, w2) - y) #dim*1
#     adagrad += gradient ** 2
#     w2 = w2 - learning_rate[1] * gradient / np.sqrt(adagrad + eps)
#     model_data['loss2'].append(loss2)
# adagrad = np.zeros([dim, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss3 = np.sqrt(np.sum(np.power(np.dot(x, w3) - y, 2))/471/12)#rmse标准差
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x.transpose(), np.dot(x, w3) - y) #dim*1
#     adagrad += gradient ** 2
#     w3 = w3 - learning_rate[2] * gradient / np.sqrt(adagrad + eps)
#     model_data['loss3'].append(loss3)
# adagrad = np.zeros([dim, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss4 = np.sqrt(np.sum(np.power(np.dot(x, w4) - y, 2))/471/12)#rmse标准差
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x.transpose(), np.dot(x, w4) - y) #dim*1
#     adagrad += gradient ** 2
#     w4 = w4 - learning_rate[3] * gradient / np.sqrt(adagrad + eps)
#     model_data['loss4'].append(loss4)
# # model_data

In [25]:
# #题目1-作图cell
# import matplotlib.pyplot as plt
 
# #读取数据
# # model_data = {
# #     't': [1,2,3],
# #     'loss': [7.1,23.2,11.8]
# # }
# plt.figure(figsize=(10,5))#设置画布的尺寸
# plt.title('Loss-Iteration Line Chart',fontsize=20)#标题，并设定字号大小
# plt.xlabel(u'x-iteration times',fontsize=14)#设置x轴，并设定字号大小
# plt.ylabel(u'y-loss',fontsize=14)#设置y轴，并设定字号大小
# plt.xlim((0, 1000))
# plt.ylim((5, 15))
# #color：颜色，linewidth：线宽，linestyle：线条类型，label：图例，marker：数据点的类型
# plt.plot(model_data['t'],model_data['loss1'],color="darkblue",linewidth=1,linestyle='--',label='learning_rate=0.5', marker='+')
# plt.plot(model_data['t'],model_data['loss2'],color="deeppink",linewidth=2,linestyle=':',label='learning_rate=2', marker='o')
# plt.plot(model_data['t'],model_data['loss3'],color="goldenrod",linewidth=1.5,linestyle='-',label='learning_rate=50', marker='*')
# plt.plot(model_data['t'],model_data['loss4'],color="green",linewidth=1.5,linestyle='-',label='learning_rate=100', marker='*')

 
# plt.legend(loc=1)#图例展示位置，数字代表第几象限
# plt.show()#显示图像

In [26]:
# # baseline-cell
# learning_rate = 100
# iter_time = 1000
# adagrad = np.zeros([dim, 1])
# eps = 0.0000000001
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse标准差
#     #每进行100次迭代, 输出loss值
#     if(t%100==0):
#         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
#     adagrad += gradient ** 2
#     w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
# np.save('weight.npy', w)
# w

In [27]:
# beat-baseline-cell
learning_rate = 1
iter_time = 1200
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    #np.dot矩阵点积, power:x^y
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse标准差
    #每进行100次迭代, 输出loss值
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    #transpose矩阵转置
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w

0:27.071214829194115
100:9.588690799485795
200:7.121841139361655
300:6.4061074254747465
400:6.15938050752305
500:6.04409754514142
600:5.973692347512183
700:5.923744683044117
800:5.885665583627625
900:5.855509409810967
1000:5.831047240421956
1100:5.810858454544786


array([[ 2.13728279e+01],
       [ 4.11497788e-01],
       [-3.80380289e-01],
       [ 7.21842933e-02],
       [-8.91467463e-01],
       [-2.25508545e-02],
       [ 3.44019091e-01],
       [-2.13831600e-01],
       [-6.56933554e-01],
       [ 1.22235755e+00],
       [-2.54816751e-01],
       [ 4.63543837e-02],
       [ 9.44164043e-02],
       [ 1.04106009e-01],
       [-7.66441610e-02],
       [-2.14618078e-02],
       [-1.26818538e-01],
       [ 1.37635432e-01],
       [ 5.39310403e-01],
       [ 7.02086106e-02],
       [-3.66774486e-02],
       [ 7.05143668e-02],
       [-1.33854887e-01],
       [ 1.44584733e-01],
       [-6.00804379e-02],
       [-1.51868955e-01],
       [ 1.02265363e-01],
       [ 4.17781144e-01],
       [-2.30133026e-01],
       [ 1.86481170e-01],
       [-9.32997343e-02],
       [ 2.47173489e-01],
       [ 1.70119978e-01],
       [-3.15460814e-01],
       [ 3.83557338e-02],
       [ 1.23310294e-01],
       [ 1.48176432e-01],
       [ 7.73059724e-02],
       [ 2.1

# **Testing**
載入 test data，並且以相似於訓練資料預先處理和特徵萃取的方式處理，使 test data 形成 240 個維度為 18 * 9 + 1 的資料。

读取csv文件,输入x每18行加一个常数项维度, 加上前9小时数据

输出矩阵weight vector与x的点乘

In [28]:
# baseline-cell
# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
testdata = pd.read_csv('/kaggle/input/ml2020spring-hw1/test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [29]:
# baseline-cell
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)

In [30]:
# baseline-cell
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

array([[ 1.        , -0.24447681, -0.24545919, ..., -0.67065391,
        -1.04594393,  0.07797893],
       [ 1.        , -1.35825331, -1.51789368, ...,  0.17279117,
        -0.10906991, -0.48454426],
       [ 1.        ,  1.5057434 ,  1.34508393, ..., -1.32666675,
        -1.04594393, -0.57829812],
       ...,
       [ 1.        ,  0.3919669 ,  0.54981237, ...,  0.26650729,
        -0.20275731,  1.20302531],
       [ 1.        , -1.8355861 , -1.8360023 , ..., -1.04551839,
        -1.13963133, -1.14082131],
       [ 1.        , -1.35825331, -1.35883937, ...,  2.98427476,
         3.26367657,  1.76554849]])

In [31]:
# # beat-baseline-cell
# mean_test_x = np.mean(test_x, axis = 0) #18 * 9 
# std_test_x = np.std(test_x, axis = 0) #18 * 9 标准差
# for i in range(len(test_x)):
#     for j in range(len(test_x[0])):
#         if std_test_x[j] != 0:
#             test_x[i][j] = (test_x[i][j] - mean_test_x[j]) / std_test_x[j]
# test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
# test_x

# **Prediction**
weight 和測試資料預測 target。

In [32]:
# baseline-cell
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

array([[ 4.85439813e+00],
       [ 1.70477089e+01],
       [ 2.46824067e+01],
       [ 8.21895055e+00],
       [ 2.78177625e+01],
       [ 2.19122801e+01],
       [ 2.32222672e+01],
       [ 3.10725644e+01],
       [ 1.73278317e+01],
       [ 5.85214354e+01],
       [ 1.42229846e+01],
       [ 1.04764815e+01],
       [ 6.13006546e+01],
       [ 5.26828547e+01],
       [ 2.21617530e+01],
       [ 1.15470080e+01],
       [ 3.17779550e+01],
       [ 6.79906030e+01],
       [-1.06195816e+00],
       [ 1.67255165e+01],
       [ 4.23555838e+01],
       [ 7.16196094e+01],
       [ 9.38624200e+00],
       [ 1.79418797e+01],
       [ 1.50841893e+01],
       [ 3.80694438e+01],
       [ 1.39718392e+01],
       [ 7.23745214e+01],
       [ 7.58179133e+00],
       [ 5.51948656e+01],
       [ 2.31500835e+01],
       [ 8.37350163e+00],
       [ 2.41499323e+00],
       [ 2.08350019e+01],
       [ 2.98997667e+01],
       [ 3.67963836e+01],
       [ 4.27930054e+01],
       [ 2.99159101e+01],
       [ 4.3

# **Save Prediction to CSV File**

In [33]:
# baseline-cell
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 4.854398134784606]
['id_1', 17.047708947804672]
['id_2', 24.68240667108498]
['id_3', 8.218950547910069]
['id_4', 27.817762495051504]
['id_5', 21.91228012532872]
['id_6', 23.222267187536055]
['id_7', 31.07256441370364]
['id_8', 17.32783173898377]
['id_9', 58.52143538754265]
['id_10', 14.222984607595171]
['id_11', 10.476481499249644]
['id_12', 61.300654609127214]
['id_13', 52.68285473726018]
['id_14', 22.161752965676015]
['id_15', 11.547008043614417]
['id_16', 31.777954977709978]
['id_17', 67.99060302672522]
['id_18', -1.0619581594624046]
['id_19', 16.72551652037045]
['id_20', 42.35558375827458]
['id_21', 71.61960936114068]
['id_22', 9.386242002279586]
['id_23', 17.941879679258786]
['id_24', 15.084189258474927]
['id_25', 38.0694438008131]
['id_26', 13.971839229333073]
['id_27', 72.37452139641543]
['id_28', 7.581791325997711]
['id_29', 55.19486556707836]
['id_30', 23.150083536305242]
['id_31', 8.37350163405719]
['id_32', 2.4149932316019282]
['id_33', 20.8350018786